# 0. Setup
Import dependencies.

In [1]:
import json
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests
from config import api_key

import pandas as pd
from sqlalchemy import create_engine

# 1. Initialize Variables
Initialize variables used in making API calls to TMDB.

## URLs
- `discover_base_url`: Base URL of TMDB's Discover API to search movies by different types of data. This includes the API key.
- `movie_base_url`: Base URL of TMDB's Movie API to search details of a specific movie by ID.
- `discover_target_url`: Full URL of TMDB's Discover API which includes TMDB properties to filter movies by. Example URL: https://api.themoviedb.org/3/discover/movie?api_key={api_key}&release_date.gte=2010-01-01&release_date.lte=2010-01-31&region=us&with_release_type=2|3&vote_count.gte=1&sort_by=popularity.desc&page=1
- `movie_target_url`: Full URL of TMDB's Movie API which includes the TMDB movie. Example URL: https://api.themoviedb.org/3/movie/550?api_key={api_key}

## TMDB Properties
- `release_date.gte={release_start_date}`: Earliest date of movies' release dates to search for using TMDB's Discover API.
- `release_date.lte={today}`: Latest date of movies' release dates to search for using TMDB's Discover API.

## Other
- `today`: Today's date.

**Hard-Coded TMDB URL Properties**:
- `region=US`: ISO 3166-1 code to filter movies' release dates by. Set specifically to *US*.
- `with_release_type=2|3`: Type of releases. *2|3* pulls movies with theatrical releases.
- `vote_count.gte=1`: Minimum number of votes by TMDB users on movies to search for in TMDB's Discover API. Set to *1* to exclude movies with 0 votes, reducing overall search results.
- `sort_by=popularity.desc`: Sort movies pulled through TMDB's Discover API by [TMDB's popularity metric](https://developers.themoviedb.org/3/getting-started/popularity) among TMDB users, descending.
- `append_to_results=credits`: Add to the result of an API call more details. This is used when making an API call to TMDB's Movie API to also add details on cast/crew of the movie.

In [29]:
today = date.today()

discover_base_url = f"https://api.themoviedb.org/3/discover/movie?api_key={api_key}"
movie_base_url = "https://api.themoviedb.org/3/movie/"

# Initialize start date as 10 years from today. Keep first day of month.
release_start_date = date((today - relativedelta(years = 10)).year, today.month, 1)
region = "US"

# Initialize variables of empty lists to populate upon iterating through APIs.
movies_lst = []
directors_lst = []
actors_lst = []

# 2. Request Data from API
Make calls to TMDB's APIs to retrieve and store data into lists.

In [30]:
# Print the range of dates within which to pull movies data from.
print(f"Request: {str(release_start_date)} to {str(today)}")

# Loop through the first 500 pages of the Discover TMDB API result. Maximum pages allowed is 500.
for page in range(1, 500):
    # Append TMDB properties, including page number, to the base URL of TMDB's Discover API.
    discover_target_url = f"{discover_base_url}&release_date.gte={str(release_start_date)}&release_date.lte={str(today)}&region=US&with_release_type=2|3&vote_count.gte=1&sort_by=popularity.desc&page={page}"

    # Retrieve 'results' from TMDB's Discover API.
    discover_response = requests.get(discover_target_url, timeout=None)
    discover_data = discover_response.json()
    results = discover_data['results']

    # Print which page the API call was made for.
    print(f"Page {page} of 500:")
    
    # For each movie in 'results' of TMDB's Discover API, make a call to TMDB's Movie API, check for criteria, and add data to lists if met.
    for movie in results:
        # Store the movie's TMDB ID.
        id = movie["id"]
        # Append ID, API key, and 'append_to_response=credits' to the base URL of TMDB's Movie API.
        movie_target_url = f"{movie_base_url}{id}?api_key={api_key}&append_to_response=credits"

        # Retrieve data from TMDB's Movie API of the particular movie.
        movie_response = requests.get(movie_target_url, timeout=None)
        movie_data = movie_response.json()
        # Store the movie's release date in a variable.
        movie_release_date = datetime.strptime(movie_data['release_date'], '%Y-%m-%d').date()

        # Check the movie's data to see whether its data should be added to our lists. If not, skip and move on to the next movie.
        if (movie_data['budget'] == 0 and movie_data['revenue'] == 0): # Skip if the movie's budget and revenue are both $0.
            print(f"{id} ({movie_data['title']}) has both $0 budget and revenue. Skipping...")
            continue
        elif movie_data['original_language'] != 'en': # Skip if the movie's original language is not English.
            print(f"{id} ({movie_data['title']}) is not originally in English. Skipping...")
            continue
        elif (movie_release_date < release_start_date or movie_release_date > today): # Skip if the movie's release date doesn't fall between the designated start date and today.
            print(f"{id} ({movie_data['title']}) was released outside of the {release_start_date} to {today} range. Skipping...")
            continue
        else: # Append movie data to our lists.
            movies_lst.append({
                "id": id,
                "title": movie_data['title'],
                "tagline": movie_data['tagline'],
                "overview": movie_data['overview'],
                "homepage": movie_data['homepage'],
                "release_date": movie_data['release_date'],
                "budget": movie_data['budget'],
                "revenue": movie_data['revenue'],
                "popularity": movie_data['popularity'],
                "runtime": movie_data['runtime'],
                "poster_path": movie_data['poster_path']
            })
            
            for actor in movie_data['credits']['cast']:
                if (actor['known_for_department'] == "Acting" and actor['character'] is not None):
                    actors_lst.append({
                        "id": actor['id'],
                        "name": actor['name'],
                        "profile_path": actor['profile_path'],
                        "movie_id": id,
                        "character": actor['character']
                    })

            for member in movie_data['credits']['crew']:
                if member['job'] != 'Director':
                    continue
                elif member['job'] == 'Director':
                    directors_lst.append({
                        "id": member['id'],
                        "name": member['name'],
                        "profile_path": member['profile_path'],
                        "movie_id": id
                    })
                    break

            print(f"{id} ({movie_data['title']}) was successfully added!")

Request: 2013-01-01 to 2023-01-14
Page 1 of 500:
315162 (Puss in Boots: The Last Wish) was successfully added!
76600 (Avatar: The Way of Water) was successfully added!
899112 (Violent Night) was successfully added!
536554 (M3GAN) was successfully added!
436270 (Black Adam) was successfully added!
661374 (Glass Onion: A Knives Out Mystery) was successfully added!
724495 (The Woman King) was successfully added!
668482 (Roald Dahl's Matilda the Musical) was successfully added!
740952 (Savage Salvation) has both $0 budget and revenue. Skipping...
505642 (Black Panther: Wakanda Forever) was successfully added!
676547 (Prey for the Devil) was successfully added!
1015963 (High Heat) has both $0 budget and revenue. Skipping...
877269 (Strange World) was successfully added!
555604 (Guillermo del Toro's Pinocchio) was successfully added!
1024546 (Detective Knight: Rogue) has both $0 budget and revenue. Skipping...
663712 (Terrifier 2) was successfully added!
551271 (Medieval) was successfully ad

Page 8 of 500:
354912 (Coco) was successfully added!
762504 (Nope) was successfully added!
156022 (The Equalizer) was successfully added!
238 (The Godfather) was released outside of the 2013-01-01 to 2023-01-14 range. Skipping...
588228 (The Tomorrow War) was successfully added!
843633 (White Elephant) has both $0 budget and revenue. Skipping...
725201 (The Gray Man) was successfully added!
744275 (After We Fell) was successfully added!
803114 (The Requin) was successfully added!
460458 (Resident Evil: Welcome to Raccoon City) was successfully added!
624860 (The Matrix Resurrections) was successfully added!
705996 (Decision to Leave) is not originally in English. Skipping...
260346 (Taken 3) was successfully added!
136799 (Trolls) was successfully added!
423108 (The Conjuring: The Devil Made Me Do It) was successfully added!
1008779 (The princess) has both $0 budget and revenue. Skipping...
454626 (Sonic the Hedgehog) was successfully added!
497698 (Black Widow) was successfully added!

480530 (Creed II) was successfully added!
68726 (Pacific Rim) was successfully added!
482321 (Ron's Gone Wrong) was successfully added!
312221 (Creed) was successfully added!
280 (Terminator 2: Judgment Day) was released outside of the 2013-01-01 to 2023-01-14 range. Skipping...
76341 (Mad Max: Fury Road) was successfully added!
764835 (The Desperate Hour) has both $0 budget and revenue. Skipping...
1023086 (Devil's Workshop) has both $0 budget and revenue. Skipping...
9479 (The Nightmare Before Christmas) was released outside of the 2013-01-01 to 2023-01-14 range. Skipping...
168259 (Furious 7) was successfully added!
539885 (Ava) was successfully added!
505262 (My Hero Academia: Two Heroes) is not originally in English. Skipping...
287947 (Shazam!) was successfully added!
619803 (The Roundup) is not originally in English. Skipping...
68721 (Iron Man 3) was successfully added!
Page 16 of 500:
817758 (TÁR) was successfully added!
76757 (Jupiter Ascending) was successfully added!
568620

790496 (Bad Luck Banging or Loony Porn) has both $0 budget and revenue. Skipping...
276907 (Legend) was successfully added!
105864 (The Good Dinosaur) was successfully added!
427641 (Rampage) was successfully added!
184345 (A Haunted House 2) was successfully added!
232672 (Blended) was successfully added!
136795 (The Heat) was successfully added!
928270 (Heatwave) has both $0 budget and revenue. Skipping...
211707 (Grounded: Making The Last of Us) has both $0 budget and revenue. Skipping...
533514 (Violet Evergarden: The Movie) is not originally in English. Skipping...
Page 23 of 500:
442249 (The First Purge) was successfully added!
307081 (Southpaw) was successfully added!
280002 (The Remaining) was successfully added!
346672 (Underworld: Blood Wars) was successfully added!
378064 (A Silent Voice: The Movie) is not originally in English. Skipping...
896448 (Beneath the Surface) has both $0 budget and revenue. Skipping...
633844 (The Deer King) has both $0 budget and revenue. Skipping

228161 (Home) was successfully added!
451915 (Beautiful Boy) was successfully added!
297761 (Suicide Squad) was successfully added!
334074 (Survivor) was successfully added!
1009921 (Reclaim) has both $0 budget and revenue. Skipping...
278927 (The Jungle Book) was successfully added!
273477 (Scouts Guide to the Zombie Apocalypse) was successfully added!
305470 (Power Rangers) was successfully added!
231001 (All Hallows' Eve) has both $0 budget and revenue. Skipping...
767304 (Redemption Day) has both $0 budget and revenue. Skipping...
Page 30 of 500:
205596 (The Imitation Game) was successfully added!
527776 (Overcomer) has both $0 budget and revenue. Skipping...
618162 (The Harder They Fall) has both $0 budget and revenue. Skipping...
454640 (The Angry Birds Movie 2) was successfully added!
1043565 (Mindcage) has both $0 budget and revenue. Skipping...
839436 (Dragon Fury) has both $0 budget and revenue. Skipping...
504253 (I Want to Eat Your Pancreas) is not originally in English. Sk

...

336850 (The Encounter) has both $0 budget and revenue. Skipping...
169860 (Hey Bartender) has both $0 budget and revenue. Skipping...
283563 (White Collar Hooligan 3) has both $0 budget and revenue. Skipping...
248888 (Questioning Darwin) has both $0 budget and revenue. Skipping...
312770 (Ana Maria in Novela Land) has both $0 budget and revenue. Skipping...
270533 (30 Years of Garbage: The Garbage Pail Kids Story) was successfully added!
150188 (Candlestick) has both $0 budget and revenue. Skipping...
662400 (Days) has both $0 budget and revenue. Skipping...
474002 (Bernard and Huey) has both $0 budget and revenue. Skipping...
348523 (Black Mass of the Nazi Sex Wizard) has both $0 budget and revenue. Skipping...
575941 (Olympic Dreams) was successfully added!
537156 (Nanny Surveillance) has both $0 budget and revenue. Skipping...
Page 473 of 500:
265347 (Chasing Ghosts) has both $0 budget and revenue. Skipping...
420635 (Beautiful Accident) has both $0 budget and revenue. Skipping...


479426 (11/8/16) has both $0 budget and revenue. Skipping...
478427 (Postcards from London) has both $0 budget and revenue. Skipping...
132868 (Uncle's Paradise) has both $0 budget and revenue. Skipping...
384289 (Peter and the Farm) has both $0 budget and revenue. Skipping...
296695 (The Black Rider: Revelation Road) has both $0 budget and revenue. Skipping...
860869 (It Came from Below) has both $0 budget and revenue. Skipping...
679205 (A Most Beautiful Thing) has both $0 budget and revenue. Skipping...
583293 (Complete Strangers) has both $0 budget and revenue. Skipping...
226792 (Complicity) has both $0 budget and revenue. Skipping...
361203 (The Middle Distance) has both $0 budget and revenue. Skipping...
302035 (Lucky Stiff) has both $0 budget and revenue. Skipping...
278377 (Whitney Cummings: I Love You) has both $0 budget and revenue. Skipping...
200504 (Cleaver Family Reunion) has both $0 budget and revenue. Skipping...
617736 (House of Cardin) has both $0 budget and revenue.

298835 (After School Massacre) has both $0 budget and revenue. Skipping...
436164 (The Hybrids Family) was successfully added!
227365 (Kiss Me) has both $0 budget and revenue. Skipping...
97607 (Gimme the Loot) has both $0 budget and revenue. Skipping...
608108 (Hallowed Ground) has both $0 budget and revenue. Skipping...
550986 (A Dangerous Idea) has both $0 budget and revenue. Skipping...
305024 (Fireman Sam: Heroes of the Storm) has both $0 budget and revenue. Skipping...
365491 (Forever) has both $0 budget and revenue. Skipping...
200725 (Highland Park) has both $0 budget and revenue. Skipping...
582350 (Sprinter) has both $0 budget and revenue. Skipping...
582148 (Mom + Mom) has both $0 budget and revenue. Skipping...
435678 (Mountain Devil) has both $0 budget and revenue. Skipping...
364281 (Stolen Dreams) has both $0 budget and revenue. Skipping...
252200 (The Sunfish) has both $0 budget and revenue. Skipping...
797003 (Dr. Gift) was successfully added!
565340 (Ms. Purple) has b

Page 490 of 500:
824739 (Brimstone Incorporated) has both $0 budget and revenue. Skipping...
446680 (Space Babes from Outer Space) has both $0 budget and revenue. Skipping...
419366 (The Messenger's Box) has both $0 budget and revenue. Skipping...
387724 (Essex Vendetta) has both $0 budget and revenue. Skipping...
378092 (Kill Kane) has both $0 budget and revenue. Skipping...
713986 (Everything But a Man) has both $0 budget and revenue. Skipping...
499963 (The Aspern Papers) has both $0 budget and revenue. Skipping...
420477 (Insomnium) has both $0 budget and revenue. Skipping...
359440 (Barista) has both $0 budget and revenue. Skipping...
705134 (Asking For It) has both $0 budget and revenue. Skipping...
622357 (Long Live Rock... Celebrate the Chaos) has both $0 budget and revenue. Skipping...
515347 (Sofía Niño de Rivera: Natural Selection) has both $0 budget and revenue. Skipping...
369761 (There's a Man in the Woods) has both $0 budget and revenue. Skipping...
323271 (Unsullied) ha

292115 (Jungle Master) has both $0 budget and revenue. Skipping...
Page 496 of 500:
257063 (Tammy Pescatelli: Finding the Funny) has both $0 budget and revenue. Skipping...
241070 (Divorce Corp.) has both $0 budget and revenue. Skipping...
409299 (Jungle Trap) has both $0 budget and revenue. Skipping...
338424 (101 Dalmatians: The Further Adventures of Thunderbolt) has both $0 budget and revenue. Skipping...
737058 (5th Borough) has both $0 budget and revenue. Skipping...
483266 (The Jade Pendant) has both $0 budget and revenue. Skipping...
436355 (Vazante) has both $0 budget and revenue. Skipping...
388374 (Rescue Dogs) has both $0 budget and revenue. Skipping...
254009 (Sucker) has both $0 budget and revenue. Skipping...
342562 (Tiny Giants 3D) has both $0 budget and revenue. Skipping...
334681 (Among the Believers) has both $0 budget and revenue. Skipping...
255496 (And Then There Was You) has both $0 budget and revenue. Skipping...
586604 (Chasing Bullitt) was successfully added!
4

In [31]:
# Check how many total movies were added to 'movies_lst'.
print(f"Total number of movies added: {len(movies_lst)}")

Total number of movies added: 2843


# 3. Load Lists into DataFrames
- Store lists of movies, directors, and actors into DataFrames.
- Save JSON files of movies, directors, and actors.

In [32]:
movies_df = pd.DataFrame(movies_lst)
movies_df.head()

,id,title,tagline,overview,homepage,release_date,budget,revenue,popularity,runtime,poster_path
0,315162,Puss in Boots: The Last Wish,Say hola to his little friends.,Puss in Boots discovers that his passion for a...,https://www.dreamworks.com/movies/puss-in-boot...,2022-12-21,90000000,205141553,9724.995,103,/1NqwE6LP9IEdOZ57NCT51ftHtWT.jpg
1,76600,Avatar: The Way of Water,Return to Pandora.,Set more than a decade after the events of the...,https://www.avatar.com/movies/avatar-the-way-o...,2022-12-16,460000000,1752000000,3003.545,192,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg
2,899112,Violent Night,You better watch out.,When a team of mercenaries breaks into a wealt...,https://www.violentnightmovie.com,2022-11-30,20000000,73000000,2254.841,112,/1XSYOP0JjjyMz1irihvWywro82r.jpg
3,536554,M3GAN,Friendship has evolved.,A brilliant toy company roboticist uses artifi...,https://www.m3ganmovie.com,2023-01-06,12000000,52580135,2067.000,102,/7CNCv9uhqdwK7Fv4bR4nmDysnd9.jpg
4,436270,Black Adam,The world needed a hero. It got Black Adam.,"Nearly 5,000 years after he was bestowed with ...",https://www.dc.com/BlackAdam,2022-10-21,200000000,389000000,1876.042,125,/pFlaoHTZeyNkG83vxsAJiGzfSsa.jpg


In [39]:
movies_df.to_json('Data/movies.json')

In [33]:
directors_df = pd.DataFrame(directors_lst)
directors_df.head()

,id,name,profile_path,movie_id
0,1450348,Joel Crawford,/3F60mZB4sGgB51jAIglwn6OrUgB.jpg,315162
1,2710,James Cameron,/9NAZnTjBQ9WcXAQEzZpKy4vdQto.jpg,76600
2,76927,Tommy Wirkola,/lhvD0FltdzJqrI5xr5IN90nyh6f.jpg,899112
3,1288796,Gerard Johnstone,/1XmsujVNMWzKQYsMSCBoM56fouW.jpg,536554
4,59521,Jaume Collet-Serra,/saAeTFfgt9YRAybXmzfmIz01UCB.jpg,436270


In [40]:
directors_df.to_json('Data/directors.json')

In [34]:
actors_df = pd.DataFrame(actors_lst)
actors_df.head()

,id,name,profile_path,movie_id,character
0,3131,Antonio Banderas,/iWIUEwgn2KW50MssR7tdPeFoRGW.jpg,315162,Puss in Boots (voice)
1,3136,Salma Hayek,/ka1luhF3DrSMEeY4dd5bKZOsPPS.jpg,315162,Kitty Softpaws (voice)
2,210172,Harvey Guillén,/yiNBonobPwqMVweB02JWufzp2l9.jpg,315162,Perrito (voice)
3,52583,Wagner Moura,/6gcfwvOueJKhDpTP9KLGuWz0Hk4.jpg,315162,Wolf (voice)
4,1373737,Florence Pugh,/421cSReX2Fktldac8SyY2k0yLwY.jpg,315162,Goldilocks (voice)


In [41]:
actors_df.to_json('Data/actors.json')

# 4. Import DataFrames to Postgres Database
Connect to the local Postgres database and store movies, directors, and actors into the respective tables. Tables within the local Postgres database should already be created before attempting.

In [35]:
# Connect to local database.
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'project03-group11_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [36]:
# Check for tables.
engine.table_names()

C:\Users\pbhan\AppData\Local\Temp\ipykernel_24200\4178942395.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['movie', 'actor', 'director']

In [42]:
# Store lists of movies, directors, and actors into Postgres as separate tables.
movies_df.to_sql(
    name='movie',
    con=engine,
    if_exists='append',
    index=False
)

843

In [43]:
directors_df.to_sql(
    name='director',
    con=engine,
    if_exists='append',
    index=False
)

840

In [44]:
actors_df.to_sql(
    name='actor',
    con=engine,
    if_exists='append',
    index=False
)

644